In [42]:
# Лабораторная работа 2 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель сети Хопфилда с дискретным состоянием
# и дискретным временем в асинхронном режиме
#
# 03.12.24
# Данный файл содержит реализацию сети Хопфилда

In [43]:
import numpy as np
from numbers import Number
from pprint import pprint

In [44]:
class Hopfield:

    def __init__(self, images, nu: float = 1) -> None:
        self.size = images.shape[1]
        self.w = np.matrix(np.zeros((self.size, self.size)))
        self.images = images
        self.neg_images = self._get_neg_images(self.images)
        self.nu = nu
        self.epochs = 0

    def _get_neg_images(self, images):
        return images * (-1)      

    def msign(self, s: Number, y_t: Number) -> Number:
        if s > 0:            
            return 1
        elif s < 0:            
            return -1
        else:            
            return y_t 

    def train(self, e=1e-6, max_iters=10000):
        '''Hopfield network train using D-projections method.
        Training process continues unless difference between old and new weights is
        small'''

        self.epochs = 0
        for _ in range(max_iters):
            old_w = self.w.copy()
            self.epochs += 1      
            for image in self.images:
                # x = np.matrix(self.images.copy())                        
                # x_plus = np.linalg.pinv(x) # pseudo-inverted matrix                      
                # w = x @ x_plus            
                # # print(w)

                x_t = np.matrix(image.copy()).T            
                self.w = self.w + (self.nu / self.size) * (x_t - (self.w @ x_t)) @ x_t.T
                np.fill_diagonal(self.w, 0)                             

                # x = np.matrix(image)            
                # self.w += (x.T @ x) / len(self.images)
            
            # Difference between old and new weights            
            w_diff = np.max(np.absolute(old_w - self.w))           
            print(f'Epoch {self.epochs}/{max_iters}: ' 
                  f'max(w{self.epochs} - w{self.epochs - 1}) = {w_diff}')
            if w_diff < e:
                break
                                                                         
        np.fill_diagonal(self.w, 0)                                
    
    def _update_neuron(self, x, neuron_idx: int) -> Number:
        return self.msign((self.w[neuron_idx] @ x).item(), x[neuron_idx])
    
    def _find_image_num(self, x, images) -> int | None:
        mask = (images == x).all(axis=1)
        search_result = np.where(mask)[0]               
        if len(search_result) > 0:
            return search_result.item()
        return None

    def predict(self, x, max_iters: int = 100):
        state = x.copy()
        relaxation_iters = 0
        for _ in range(max_iters):            
            relaxation_iters += 1
            prev_state = state.copy()
            for i in range(self.size):                
                state[i] = self._update_neuron(state, i)
            if np.array_equal(prev_state, state):
                is_negative = False
                image_num = self._find_image_num(state, self.images)                
                neg_image_num = self._find_image_num(state, self.neg_images)
                is_negative = True if neg_image_num is not None else False

                return (relaxation_iters, state,
                        (image_num if image_num is not None else neg_image_num),
                        is_negative)            
        return max_iters, state, None, None

In [45]:
def is_negative(image, neg_image) -> bool:
    return(np.array_equal(image, neg_image))

In [46]:
def image_beautiful_print(image, rows, cols):
    image = image.astype(np.object_)
    image[image == 1] = '⬜'
    image[image == -1] = '⬛'
    image = image.reshape(rows, cols)
    image_list = image.tolist()
    for row in image_list:
        print(''.join(row))

In [47]:
alphabet = np.array(
    [
        [-1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1],
        [1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, 1, 1, -1],
        [-1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1],
        [-1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1],      
    ]
)
for a in alphabet:
    image_beautiful_print(a, 4, 4)
    print()
network = Hopfield(alphabet, 0.8)
network.train()

⬛⬜⬜⬛
⬜⬛⬛⬜
⬜⬜⬜⬜
⬜⬛⬛⬜

⬜⬜⬜⬛
⬜⬛⬛⬜
⬜⬛⬛⬜
⬜⬜⬜⬛

⬛⬜⬜⬛
⬜⬛⬛⬜
⬜⬛⬛⬜
⬛⬜⬜⬛

⬛⬜⬜⬜
⬜⬛⬛⬛
⬜⬛⬛⬛
⬛⬜⬜⬜

Epoch 1/10000: max(w1 - w0) = 0.18335625
Epoch 2/10000: max(w2 - w1) = 0.13612411816406253
Epoch 3/10000: max(w3 - w2) = 0.10281350972598263
Epoch 4/10000: max(w4 - w3) = 0.0867712706257644
Epoch 5/10000: max(w5 - w4) = 0.07447128395611974
Epoch 6/10000: max(w6 - w5) = 0.06435094634741678
Epoch 7/10000: max(w7 - w6) = 0.05592150923059058
Epoch 8/10000: max(w8 - w7) = 0.048766149844073015
Epoch 9/10000: max(w9 - w8) = 0.04260792622064202
Epoch 10/10000: max(w10 - w9) = 0.037264359040410255
Epoch 11/10000: max(w11 - w10) = 0.03260718427769249
Epoch 12/10000: max(w12 - w11) = 0.02853904388066364
Epoch 13/10000: max(w13 - w12) = 0.024981431292904532
Epoch 14/10000: max(w14 - w13) = 0.021868560854482633
Epoch 15/10000: max(w15 - w14) = 0.019144106028547325
Epoch 16/10000: max(w16 - w15) = 0.016759294167454253
Epoch 17/10000: max(w17 - w16) = 0.014671654864436223
Epoch 18/10000: max(w18 - w17) = 0.01284410330

In [48]:
test_image = np.array([-1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1])
r_iters, state, image_idx, is_negative = network.predict(test_image, 100)
if image_idx:
    predicted_img = network.neg_images[image_idx] if is_negative else network.images[image_idx]
else:
    predicted_img = state
print(f'Prediction success? - {True if image_idx is not None else False}')
print(f'Image number (index begins from 0): {image_idx}.\nImage: {predicted_img}.\nIters for relax: {r_iters}')
print(f'Is this image a negative? - {is_negative}')
print('ORIGINAL IMAGE')
image_beautiful_print(test_image, 4, 4)
print('RECOVERED IMAGE')
image_beautiful_print(predicted_img, 4, 4)
print('NEGATIVE OF RECOVERED IMAGE')
image_beautiful_print(predicted_img * (-1), 4, 4)

Prediction success? - True
Image number (index begins from 0): 0.
Image: [-1  1  1 -1  1 -1 -1  1  1  1  1  1  1 -1 -1  1].
Iters for relax: 1
Is this image a negative? - False
ORIGINAL IMAGE
⬛⬜⬜⬛
⬜⬛⬛⬜
⬜⬜⬜⬜
⬜⬛⬛⬜
RECOVERED IMAGE
⬛⬜⬜⬛
⬜⬛⬛⬜
⬜⬜⬜⬜
⬜⬛⬛⬜
NEGATIVE OF RECOVERED IMAGE
⬜⬛⬛⬜
⬛⬜⬜⬛
⬛⬛⬛⬛
⬛⬜⬜⬛
